<a href="https://colab.research.google.com/github/abriciof/sistema-neuro-simbolico/blob/main/sistema_neuro_simbolico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🪢 Implementação do método de Aprendizagem Relacional no paradigma de Sistemas Neuro-Simbólicos

*Universidade Federal do Amazonas*<br>
*Engenharia da Computação*<br>
*Inteligência Artificial Turma EC01 2021/1*<br>

*   Fabrício da Costa Guimarães - 21950515
*   Laura Aguiar Martinho - 21952064
*   Lorena Bastos Amazonas - 21952638
*   Mattheus Smith Costa - 21954379

## Bibliotecas

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from math import floor, ceil
from pylab import rcParams
from time import gmtime, strftime

TensorFlow 1.x selected.


## Importando e separando a base 
Necessário: *(base-positivos-negativos.csv)*

In [2]:
# Codificar one-hot
def encode(series):
    return  pd.get_dummies(series.astype(str))

# Lendo o csv da base
df = pd.read_csv('/content/base-positivos-negativos.csv')
dados = df.sample(frac=1).reset_index(drop=True)

# Predicados
predicados = dados.drop('hd(X)',1)

# Labels
labels = encode(dados['hd(X)'])

# Separando a base entre treino 90% e teste 10%
tam_treino = 0.9
cnt_treino = floor(predicados.shape[0] * tam_treino)
x_treino = predicados.iloc[0:cnt_treino].values
y_treino = labels.iloc[0:cnt_treino].values
x_teste =  predicados.iloc[cnt_treino:].values
y_teste = labels.iloc[cnt_treino:].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


## Estrutura da Rede Neural Artificial

In [4]:
taxa_aprendizado = 0.01
X = tf.placeholder(tf.float32, shape=(None, x_treino.shape[1]), name="X")
y = tf.placeholder(tf.int64,shape=(None,2), name="y")

# Estrutura da Rede Neural Artificial
with tf.name_scope("rn"):
    camada_1 = fully_connected(X, x_treino.shape[1], scope="camada_1")
    camada_2 = fully_connected(camada_1, predicados.shape[0], scope="camada_2")
    saida = fully_connected(camada_2, 2, scope="saida", activation_fn=None)

# Operações de Treino
with tf.name_scope("fcusto"):
    y_float = tf.cast(y, tf.float32)
    sigmoidtropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_float, logits=saida)
    custo = tf.reduce_mean(sigmoidtropy, name="custo")

with tf.name_scope("treino"):
    otimizador = tf.train.GradientDescentOptimizer(taxa_aprendizado)
op_treino = otimizador.minimize(custo)

with tf.name_scope("eval"):
    previsao_correta = tf.equal(tf.argmax(saida,1), tf.argmax(y,1))
eficiencia = tf.reduce_mean(tf.cast(previsao_correta, "float"))

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Treino e testes

In [33]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epocas = 15

with tf.Session() as sess:
  init.run()
  for epoca in range(n_epocas):
    particao_total = int(len(x_treino) / 1)
    x_parts = np.array_split(x_treino, particao_total)
    y_parts = np.array_split(y_treino, particao_total)
    for i in range(particao_total):
      part_x, part_y = x_parts[i], y_parts[i]
      sess.run(op_treino,feed_dict={X: part_x, y: part_y})
      acc_treino = eficiencia.eval(feed_dict={X: part_x, y: part_y})
      acc_teste = eficiencia.eval(feed_dict={X: x_teste, y: y_teste})
    if epoca % 1 == 0:
        print("Época:", epoca+1,"- Acurácia teste:" ,acc_teste)
    x_t = np.array_split(x_teste, x_teste.shape[0])
    y_t = np.array_split(y_teste, y_teste.shape[0])
  for k in range(len(x_t)):
      x_t2, y_t2 = x_t[k], y_t[k]
      predicao = ''
      label = ''

      if (y_t2.item(0) == 1):
        label = 'falso'
      elif(y_t2.item(1) == 1):
        label = 'verdadeiro'

      pred = saida.eval(feed_dict={X:x_t2})
      if (pred.item(0) > pred.item(1)):
        predicao = 'falso'
      elif(pred.item(1) > pred.item(0)):
        predicao = 'verdadeiro'

      print()
      print(f'\n{x_t2}')
      print(f'predição -> {pred} {predicao}')
      print(f'label -> {y_t2} {label}')

Época: 1 - Acurácia teste: 0.6
Época: 2 - Acurácia teste: 0.6
Época: 3 - Acurácia teste: 0.6
Época: 4 - Acurácia teste: 0.6
Época: 5 - Acurácia teste: 0.6
Época: 6 - Acurácia teste: 0.6
Época: 7 - Acurácia teste: 0.6
Época: 8 - Acurácia teste: 0.6
Época: 9 - Acurácia teste: 0.8
Época: 10 - Acurácia teste: 0.8
Época: 11 - Acurácia teste: 1.0
Época: 12 - Acurácia teste: 1.0
Época: 13 - Acurácia teste: 1.0
Época: 14 - Acurácia teste: 1.0
Época: 15 - Acurácia teste: 1.0


[[1 0 0 1 1 0 0 0 0 0 0 1]]
predição -> [[-0.42527854  0.3641236 ]] verdadeiro
label -> [[0 1]] verdadeiro


[[0 1 0 1 1 0 0 0 1 1 0 0]]
predição -> [[-0.73514104  0.5966549 ]] verdadeiro
label -> [[0 1]] verdadeiro


[[1 0 0 1 0 1 0 0 1 1 1 0]]
predição -> [[ 0.17873408 -0.27969512]] falso
label -> [[1 0]] falso


[[0 1 0 1 1 0 1 1 0 0 0 0]]
predição -> [[-1.1805335  0.9256139]] verdadeiro
label -> [[0 1]] verdadeiro


[[0 1 0 1 0 1 1 1 0 0 0 0]]
predição -> [[ 0.51593375 -0.31528124]] falso
label -> [[1 0]] falso
